## Imports 

In [41]:
!pip install folium==0.8.2

In [42]:
!pip install requests==v2.21.0

In [43]:
import folium
import operator
import requests
import json
import pandas as pd
from os import walk
from unicodedata import normalize

In [44]:
data = pd.read_excel('data/table_of_wells_april_2018.xlsx', sheet_name='Plan1')

In [45]:
data = data.drop({'CADASTRO', 'OPERADOR', 'POCO_OPERADOR', 'BLOCO', 'SIG_CAMPO', 'CAMPO', 'TERRA_MAR', 
                'POCO_POS_ANP', 'LATITUDE_BASE_4C', 
                'LONGITUDE_BASE_4C', 'DATUM_HORIZONTAL', 'TIPO_DE_COORDENADA_DE_BASE', 'DIRECAO', 'PROFUNDIDADE_VERTICAL_M',
                'PROFUNDIDADE_SONDADOR_M','PROFUNDIDADE_MEDIDA_M', 'REFERENCIA_DE_PROFUNDIDADE', 'MESA_ROTATIVA', 
                'COTA_ALTIMETRICA_M','LAMINA_D_AGUA_M', 'DATUM_VERTICAL', 'UNIDADE_ESTRATIGRAFICA', 'GEOLOGIA_GRUPO_FINAL',
                'GEOLOGIA_FORMACAO_FINAL', 'GEOLOGIA_MEMBRO_FINAL', 'CDPE', 'AGP', 'PC', 'PAG', 'PERFIS_CONVENCIONAIS',
                'DURANTE_PERFURACAO', 'PERFIS_DIGITAIS', 'PERFIS_PROCESSADOS', 'PERFIS_ESPECIAIS', 'AMOSTRA_LATERAL', 
                'SISMICA', 'TABELA_TEMPO_PROFUNDIDADE', 'DADOS_DIRECIONAIS', 'TESTE_A_CABO', 'CANHONEIO', 'TESTEMUNHO', 
                'GEOQUIMICA', 'SIG_SONDA', 'NOM_SONDA', 'DHA_ATUALIZACAO'}, 1)

In [46]:
def get_produtors(dfs):
    '''
    Filters producting wells in a field
    '''
    productor = dfs[(dfs['TIPO']==u'Explotatório') & 
                    (dfs['SITUACAO']=='PRODUZINDO') & 
                    (dfs['CATEGORIA']=='Desenvolvimento') & 
                    (dfs['RECLASSIFICACAO']==u'PRODUTOR COMERCIAL DE PETRÓLEO')]
    productor.reset_index(inplace=True)
    return productor

In [47]:
def get_injections(dfs):
    '''
    Filters the injection wells in a field
    '''
    injector = dfs[(dfs['TIPO']==u'Explotatório') & 
                   (dfs['SITUACAO']=='INJETANDO') &
                   (dfs['CATEGORIA']==u'Injeção') & 
                   (dfs['RECLASSIFICACAO']==u'INJEÇÃO DE ÁGUA')]
    injector.reset_index(inplace=True)
    return injector

In [48]:
produtor = get_produtors(data)
injetor = get_injections(data)
pai = pd.concat([produtor, injetor])
pai.reset_index(inplace=True)
pai = pai.drop({'level_0', 'index'}, 1)

In [49]:
pocos = pai["POCO"].tolist()
pocos = [poco.upper().replace(' ', '').replace('-', '') for poco in pocos]
pai["POCO"] = pocos

In [50]:
pai.head(10)

,POCO,ESTADO,BACIA,TIPO,CATEGORIA,RECLASSIFICACAO,SITUACAO,INICIO,TERMINO,CONCLUSAO,TITULARIDADE,LATITUDE_BASE_DD,LONGITUDE_BASE_DD,TESTE_DE_FORMACAO
0,7SZ514SE,SE,Sergipe,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2009-10-29,2009-11-06,2009-11-06,Público,"-10,6621772222","-37,0976913888",NaN
1,7MLL44HPRJS,RJ,Campos,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2009-11-21,2009-12-27,2010-02-16,Público,"-22,4442986111","-39,9418836111",NaN
2,7FP398RN,RN,Potiguar,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2009-10-26,2009-10-30,2009-12-18,Público,"-5,2330747222","-36,5125927777",NaN
3,7FP375RN,RN,Potiguar,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2009-03-19,2009-03-23,2009-07-20,Público,"-5,2741797222","-36,6021188888",NaN
4,7FP376RN,RN,Potiguar,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2009-04-13,2009-04-18,2009-06-17,Público,"-5,2615075","-36,5694911111",NaN
5,7FP382RN,RN,Potiguar,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2009-02-18,2009-02-23,2009-03-20,Público,"-5,2615577777","-36,5730975",NaN
6,7ARG753RN,RN,Potiguar,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2009-12-11,2009-12-12,2009-12-28,Público,"-5,3232283333","-36,7376497222",NaN
7,7FP399RN,RN,Potiguar,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2010-01-27,2010-02-01,2010-02-26,Público,"-5,2339658333","-36,5098758333",NaN
8,7ET1019RN,RN,Potiguar,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2009-01-15,2009-01-16,2009-02-15,Público,"-5,3818388888","-36,869935",NaN
9,7ET1002RN,RN,Potiguar,Explotatório,Desenvolvimento,PRODUTOR COMERCIAL DE PETRÓLEO,PRODUZINDO,2008-11-16,2008-11-17,2008-11-22,Público,"-5,3864588888","-36,8663538888",NaN


### Mapa com todos os pocos exploratórios

In [51]:
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/vnd.geo+json, application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

In [52]:
geo_json_br = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/?resolucao=2&formato=application/vnd.geo+json",headers=headers).json()

#### Quais são os estados que mais possuem/possuíram pocos?

In [53]:
oil_wells_per_state = dict(data["ESTADO"].value_counts())

In [54]:
oil_wells_per_state

{'RN': 8162,
 'BA': 7121,
 'SE': 4672,
 'RJ': 3568,
 'ES': 2250,
 'CE': 1273,
 'AL': 969,
 'AM': 534,
 'MA': 316,
 'SP': 292,
 'PA': 102,
 'AP': 84,
 'PR': 75,
 'MG': 56,
 'SC': 55,
 'RS': 25,
 'MS': 19,
 'MT': 12,
 'PI': 6,
 'PE': 6,
 'AC': 6,
 'PB': 5,
 'GO': 2,
 'RR': 2}

In [55]:
rank_oil_wells = sorted(oil_wells_per_state.items(), key=operator.itemgetter(1),reverse=True)

In [56]:
rank_oil_wells[:5]

[('RN', 8162), ('BA', 7121), ('SE', 4672), ('RJ', 3568), ('ES', 2250)]

In [57]:
states_br_json = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados").json()

In [58]:
states_br_json

[{'id': 11,
  'sigla': 'RO',
  'nome': 'Rondônia',
  'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}},
 {'id': 12,
  'sigla': 'AC',
  'nome': 'Acre',
  'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}},
 {'id': 13,
  'sigla': 'AM',
  'nome': 'Amazonas',
  'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}},
 {'id': 14,
  'sigla': 'RR',
  'nome': 'Roraima',
  'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}},
 {'id': 15,
  'sigla': 'PA',
  'nome': 'Pará',
  'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}},
 {'id': 16,
  'sigla': 'AP',
  'nome': 'Amapá',
  'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}},
 {'id': 17,
  'sigla': 'TO',
  'nome': 'Tocantins',
  'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}},
 {'id': 21,
  'sigla': 'MA',
  'nome': 'Maranhão',
  'regiao': {'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}},
 {'id': 22,
  'sigla': 'PI',
  'nome': 'Piauí',
  'regiao': {'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}},
 {'id': 23,
  'sigla': 'CE',
  'nome': 'Ceará',
  'r

In [59]:
id_state,initial_state,name_state,oil_wells, exploratory = [],[],[],[],[]
for state in states_br_json:
    sigla = state['sigla']
    id_state.append(state['id'])
    initial_state.append(sigla)
    name_state.append(state['nome'])
    if state['sigla'] in oil_wells_per_state:
        oil_wells.append(oil_wells_per_state[sigla])
    else:
        oil_wells.append(0)
    oil_found = list(pai[pai['ESTADO'] == sigla].ESTADO.value_counts())
    exploratory.append(0 if(len(oil_found) == 0) else oil_found[0])
    
states_wells = {'id':id_state,'initial':initial_state,'name':name_state,
                'oil_wells':oil_wells,'exploratory':exploratory}

In [60]:
df_states = pd.DataFrame.from_dict(states_wells)

In [61]:
df_states

,id,initial,name,oil_wells,exploratory
0,11,RO,Rondônia,0,0
1,12,AC,Acre,6,0
2,13,AM,Amazonas,534,2
3,14,RR,Roraima,2,0
4,15,PA,Pará,102,0
5,16,AP,Amapá,84,0
6,17,TO,Tocantins,0,0
7,21,MA,Maranhão,316,0
8,22,PI,Piauí,6,0
9,23,CE,Ceará,1273,676


In [62]:
df_states.set_index('id',inplace=True)
df_states.index.name = None
df_states["codarea"] = df_states.index
df_states.head()

,initial,name,oil_wells,exploratory,codarea
11,RO,Rondônia,0,0,11
12,AC,Acre,6,0,12
13,AM,Amazonas,534,2,13
14,RR,Roraima,2,0,14
15,PA,Pará,102,0,15


In [63]:
makeStateRequest = lambda x:requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/"+str(x)+"/"+
                        "?formato=application/vnd.geo+json&resolucao=1",headers=headers).json()

In [64]:
each_state = []
for i in df_states.index:
    each_state+=(makeStateRequest(int(i))['features'])
each_state

[{'type': 'Feature',
  'properties': {'codarea': '11',
   'centroide': [-62.84169849643017, -10.91332465228205]},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-62.7943, -8.0275],
     [-62.7862, -8.0263],
     [-62.738, -8.05],
     [-62.7331, -8.0565],
     [-62.6788, -8.1114],
     [-62.6855, -8.1164],
     [-62.673, -8.1453],
     [-62.6866, -8.1737],
     [-62.6598, -8.1996],
     [-62.655, -8.229],
     [-62.6, -8.2722],
     [-62.5619, -8.284],
     [-62.5611, -8.3138],
     [-62.5395, -8.3581],
     [-62.5184, -8.3847],
     [-62.4789, -8.3456],
     [-62.4654, -8.3397],
     [-62.4397, -8.3678],
     [-62.4245, -8.3749],
     [-62.384, -8.3775],
     [-62.3608, -8.3985],
     [-62.3692, -8.4461],
     [-62.366, -8.4935],
     [-62.3539, -8.5117],
     [-62.3373, -8.5166],
     [-62.3258, -8.5637],
     [-62.341, -8.6028],
     [-62.2895, -8.639],
     [-62.2793, -8.6291],
     [-62.2943, -8.6054],
     [-62.2893, -8.5889],
     [-62.2688, -8.5757],
     [-62.2286, -8.

In [65]:
geo_states_json = {'type':'FeatureCollection','features':each_state}

In [66]:
for state in geo_states_json['features']:
    id_aux = int(state['properties']['codarea'])
    state['properties']['oil_wells'] = str(df_states.loc[id_aux,"oil_wells"])
    state['properties']['name'] = str(df_states.loc[id_aux,"name"])
    state['properties']['exploratory'] = str(df_states.loc[id_aux,"exploratory"])

In [67]:
m = folium.Map(
    location=[-13.702797,-69.6865109],
    zoom_start=4,
    tiles='Stamen Terrain'
)  

In [68]:
from branca.colormap import linear

colormap = linear.YlGn_03.scale(
    df_states.oil_wells.min(),
    df_states.oil_wells.max())

colormap.caption="#Oil Wells on Brazil"

print(colormap(5000.0))

colormap

#91d081


In [69]:
folium.GeoJson(geo_states_json,
               name='oil_wells',
               style_function=lambda x: {'fillColor': colormap(df_states.loc[int(x['properties']['codarea']),
                                                                           "oil_wells"]),
                                         'color': 'black','weight':2, 'fillOpacity':0.8},
               tooltip=folium.GeoJsonTooltip(fields=['name',"oil_wells"], 
                                            aliases=['Name:',"Amount Oil Wells:"], 
                                            localize=True)
              ).add_to(m)

# Add a LayerControl.
folium.LayerControl().add_to(m)

# And the Color Map legend.
colormap.add_to(m)

m

### Produção por poço

In [70]:
files = []
for (dirpath, dirnames, filenames) in walk('data/production_well'):
    for filename in filenames:
        if filename.endswith('.xls') or filename.endswith('.xlsx'):
            files.append('%s/%s' % (dirpath, filename))

In [71]:
wells = []
for file in sorted(files):
    excel_file = pd.read_excel(file)
    initial_column = excel_file.columns[0]

    period_value = None
    well_data = None
    production_data = None
    try:
        if initial_column.startswith('ANP - Agência Nacional do Petroleo'):
            data = pd.read_excel(file, skiprows=4, header=[0, 1], index_col=None)

            period = data.xs('Período', axis=1, level=0, drop_level=True).iloc[:, 0]
            period = period.dropna().unique()
            period = period[0].replace('/', '-')

            production_col = None
            field_col = None
            for column in data.columns:
                if column[0].startswith('Petróleo (bbl/dia)'):
                    production_col = column
                if column[0].startswith('Campo'):
                    field_col = column
            selected_data = data[[('Nome Poço', 'ANP'), field_col, production_col]]

            period_value = period
            well_data = selected_data[('Nome Poço', 'ANP')].values
            field_data = selected_data[field_col].values
            production_data = selected_data[production_col].values

        elif initial_column.startswith('COD_POCO') or initial_column.startswith('NOM_POCO_ANP'):
            data = pd.read_excel(file)

            period = data['PERIODO'].dropna().unique()
            period = period[0].replace('_', '-')

            selected_data = data[['NOM_POCO_ANP', 'NOM_CAMPO', 'OLEO_BBL_DIA']]

            well_data = pd.DataFrame(columns=['well', 'production_%s' % period])

            period_value = period
            well_data = data['NOM_POCO_ANP'].values
            field_data = data['NOM_CAMPO'].values
            production_data = data['OLEO_BBL_DIA'].values

    except Exception as exc:
        print('********** Exception **********', exc)

    well_df = pd.DataFrame(columns=['well', 'field', '%s-01' % period])
    well_df['well'] = well_data
    well_df['field'] = field_data
    well_df['%s-01' % period] = pd.to_numeric(production_data)

    # Remove nan in well index.
    well_df = well_df.drop(well_df.loc[well_df.well.isnull()].index, axis=0)
    # Remove nan in field index.
    well_df = well_df.drop(well_df.loc[well_df.field.isnull()].index, axis=0)
    # Remove description index.
    well_df = well_df.drop(well_df.loc[well_df.well.map(len) > 50].index, axis=0)

    well_df.well = well_df.well.map(str.strip)
    well_df.well = well_df.well.map(str.lower)
    well_df.field = well_df.field.map(str.strip)
    well_df.field = well_df.field.map(str.lower)

    def normalize_str(input_str):
        return normalize('NFKD', input_str).encode('ascii', 'ignore').decode('utf-8')
    well_df.well = well_df.well.map(normalize_str)
    well_df.field = well_df.field.map(normalize_str)

    def replace_str(input_str):
        return input_str.replace(' ', '').replace('-', '')
    well_df.well = well_df.well.map(replace_str)
    well_df.field = well_df.field.map(replace_str)
    
    well_df = well_df.dropna()
    well_df['index'] = well_df.apply(lambda row: '%s' % (row['well'].upper()), axis=1)

    well_df = well_df.drop('well', axis=1)
    well_df = well_df.drop('field', axis=1)
    well_df = well_df.set_index('index')

    well_df = well_df[~well_df.index.duplicated(keep='first')]
    
    wells.append(well_df) 

df_oil_well_salt = pd.concat(wells, axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [72]:
df_oil_well_salt = df_oil_well_salt.fillna(0)

In [73]:
df_oil_well_salt

,2016-01-01,2016-02-01,2016-03-01,2016-04-01,2016-05-01,2016-06-01,2016-07-01,2016-08-01,2016-09-01,2016-10-01,...,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01
1BRSA108AESS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1BRSA1116RJS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,222.1723,0.0000,0.0000,...,3033.0093,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1BRSA1146RJS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1BRSA594SPS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1BRSA976RJS,0.0000,4038.4289,0.0000,0.0000,16793.2997,19357.6807,17324.6283,19303.0414,421.0546,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2ANP1RJS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3BRSA1017DRJS,3217.8084,940.7094,1039.1168,1645.0457,1578.5540,1555.8059,1554.5046,1668.1607,1594.7409,1728.0598,...,2858.4587,3107.4003,3024.9240,2992.8340,1745.1677,586.4773,711.2949,693.0370,504.9356,683.4728
3BRSA1053RJS,3444.0484,13159.8767,20.1903,0.0000,3389.5847,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3BRSA1054DRJS,3089.7979,3020.2906,3049.9899,408.6213,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3BRSA1064RJS,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [74]:
oil_wells_production_sum = { index:sum(row) for index,row in df_oil_well_salt.iterrows()}

In [75]:
ordered_wells_production_sum = sorted(oil_wells_production_sum.items(), key=operator.itemgetter(1),reverse=True) 

In [76]:
ordered_wells_production_sum[:10]

[('7SPH1SPS', 974879.0969),
 ('7SPH7DSPS', 964067.8527999999),
 ('9LL20DRJS', 883836.1658999999),
 ('7LL27RJS', 865456.6222000001),
 ('9LL2RJS', 862124.3006000001),
 ('7JUB34HESS', 859319.6207000003),
 ('9LL12DRJS', 821162.3254000001),
 ('7LL28DRJS', 815882.1224999997),
 ('7LL31DRJS', 803412.6642),
 ('8LL81DRJS', 775180.3458999998)]

In [77]:
pai['LATITUDE_BASE_DD'] = [float(i.replace(",",".")) for i in list(pai['LATITUDE_BASE_DD'])]

In [78]:
pai['LONGITUDE_BASE_DD']= [float(i.replace(",",".")) for i in list(pai['LONGITUDE_BASE_DD'])]

In [79]:
five_oil_productors = []
five_oil_names = set([i[0] for i in ordered_wells_production_sum])
for index,row in pai.iterrows():
    if(row['POCO'] in five_oil_names):
        folium.Marker([row['LATITUDE_BASE_DD'],row['LONGITUDE_BASE_DD']],popup='<i>'+row['POCO']+'</i>').add_to(m)

In [80]:
m